In [1]:
import os
import random
import numpy as np
import cv2
import json
import pickle
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm.notebook import tqdm

## Settings

In [2]:
# set the size of the dataset, manually
samples = 10000
# set the target orientation
orientation = 1
# set the scale for resizing the images
scale = .5

## Helper Functions

In [3]:
# function for reading the Json files
def idx_finder(folder):
    '''
    This function takes the folder name (string),
    and returns a list of indices of the target
    orientation and valid frames in the folder.
    '''
    # set the specific folder's path
    path = 'Data_MIT/' + folder

    # read the json files
    # ------------------------------------------------------------------
    # read the Face Crop json
    with open(path + '/appleFace.json', 'r') as file:
        face = json.load(file)
    # read the Left Eye json
    with open(path + '/appleLeftEye.json', 'r') as file:
        eye_l = json.load(file)
    # read the Right Eye json
    with open(path + '/appleRightEye.json', 'r') as file:
        eye_r = json.load(file)
    # read the Scree json
    with open(path + '/screen.json', 'r') as file:
        screen = json.load(file)
    
    # make a list of all valid indices
    # ------------------------------------------------------------------
    # list of all frames with full face in it
    faces = [idx for idx, val in enumerate(face['IsValid']) if val == 1]
    # list of all frames with the left eye in it
    eyes_l = [idx for idx, val in enumerate(eye_l['IsValid']) if val == 1]
    # list of all frames with the right eye in it
    eyes_r = [idx for idx, val in enumerate(eye_r['IsValid']) if val == 1]
    # list of all portrait frames
    portraits = [idx for idx, ori in enumerate(screen['Orientation']) if ori == orientation]
    # intersection of all those lists
    indices = [idx for idx in faces if idx in eyes_l and idx in eyes_r and idx in portraits]
    
    return indices
    
    
    
# function for getting the gaze coordinates
def get_labels(folder, indices):
    '''
    The function takes the folder name (string),
    and a list of indices (list) and returns an
    np.array of gaze coordinates or the labels.
    '''
    # set the specific folder's path
    path = 'Data_MIT/' + folder
    
    # read the dot json
    with open(path + '/dotInfo.json', 'r') as file:
        dot = json.load(file)
    
    # list of XCam coordinate for valid frames
    dot_x = [x for idx, x in enumerate(dot['XCam']) if idx in indices]
    # list of YCam coordinate for valid frames
    dot_y = [y for idx, y in enumerate(dot['YCam']) if idx in indices]
    # make an np.array out of the coordinates for valid frames
    dots = np.column_stack((dot_x, dot_y))
    
    return dots
    
    

## Picking Random Frames

In [6]:
# make a list of the folders in the data directory
folders = [name for name in os.listdir('Data_MIT/')]
# drop the irrelevant file in the data directory
folders.remove('LICENSE.md')
folders.remove('README.md')

# create a list of valid and portrait frames in folders
frames_all = []
for folder in tqdm(folders):
    # make a list of valid frames
    indices = idx_finder(folder)
    frames = [folder+str(idx) for idx in indices]
    frames_all = frames_all + frames
    break